Démonstration

In [13]:
from threading import Thread 
import time
def simple_hello_world():
    time.sleep(5)
    print("hello world")
    
t1 = Thread(target=simple_hello_world)
t1.start()

hello world


In [11]:
from threading import Thread 

def simple_hello_world():
    print("hello world")
    
t1 = Thread(target=simple_hello_world)
t1.start()
try:
    t1.start()
except RuntimeError as e:
    print(f"\n{e}")

hello world
threads can only be started once



In [15]:
def info(thread):
    print(thread.name)
    print(thread.ident)
    print(thread.is_alive())

t1 = Thread(target=simple_hello_world)
t1.start()
info(t1)

t2 = Thread(target=simple_hello_world)
t2.start()
info(t2)

Thread-18 (simple_hello_world)
38628
True
Thread-19 (simple_hello_world)
10044
True
hello world
hello world


In [17]:
def print_to_console(caractere):
    for i in range(0, 10):
        print(caractere, end="", flush=True)
        time.sleep(2)
thread_asterix = Thread(target=print_to_console, args="*")
thread_point = Thread(target=print_to_console, args=".")
thread_asterix.start()
time.sleep(10)
thread_point.start()

*****.*.*.*.*.*.....

In [20]:
def print_to_console(caractere):
    for i in range(0, 10):
        print(caractere, end="", flush=True)
        time.sleep(1)
thread_asterix = Thread(target=print_to_console, args="*")
thread_point = Thread(target=print_to_console, args=".")
thread_asterix.start()

thread_asterix.join()
thread_point.start()

**********..........

In [24]:
class CustomThread(Thread):
    def __init__(self, caractere = "", name = None, daemon = None, target = None):
        super().__init__(name = name, daemon = daemon, target = target)
        self.name = name
        self.caractere = caractere
        
    def run(self):
        print(self.name)
        for i in range(0, 10):
            print(self.caractere, end="", flush=True)
            time.sleep(1)
thread_asterix = CustomThread("*", name = "Asterix")
thread_point = CustomThread(".", name = "Point")
thread_asterix.start()
thread_point.start()
    

AsterixPoint
.
*.**..**..**.*..**.

Démonstration du besoin de mémoire local par thread

In [35]:
import threading
import time
import random
from threading import Thread

class DataPartagee:
    value = None
    
def show_value(data):
    try:
        val = data.value
    except AttributeError:
        print(threading.current_thread().name, " Value n'est pas assignée")
    else:
        print(threading.current_thread().name, " Value vaut ", val)

def worker(data):
    #print(f"{threading.current_thread().name} globals : {list(globals().keys())}")
    show_value(data)
    data.value = random.randint(0, 10)
    show_value(data)
    time.sleep(2)
    show_value(data)

print(threading.current_thread(), "- Starting ")

data = DataPartagee()

for i in range(2):
    t = Thread(name = "T" +str(i+1), target=worker, args=(data,))
    t.start()    

print(threading.current_thread(), " -  Se Termine")

<_MainThread(MainThread, started 10056)> - Starting 
T1  Value vaut  None
T1  Value vaut  8
T2  Value vaut  8
T2  Value vaut  0
<_MainThread(MainThread, started 10056)>  -  Se Termine
T1  Value vaut  0
T2  Value vaut  0


Démonstration de la mémoire local par thread


In [39]:
import threading
import time
import random
from threading import Thread

class DataPartagee:
    value = None
    
def show_value(data):
    try:
        val = data.value
    except AttributeError:
        print(threading.current_thread().name, " Value n'est pas assignée")
    else:
        print(threading.current_thread().name, " Value vaut ", val)

def worker(data):
    #print(f"{threading.current_thread().name} globals : {list(globals().keys())}")
    show_value(data)
    data.value = random.randint(0, 10)
    show_value(data)
    time.sleep(2)
    show_value(data)

print(threading.current_thread(), "- Starting ")

local_data = threading.local()

for i in range(2):
    t = Thread(name = "T" +str(i+1), target=worker, args=(local_data,))
    t.start()

worker(local_data)

print(threading.current_thread(), " -  Se Termine")

<_MainThread(MainThread, started 10056)> - Starting 
T1  Value n'est pas assignée
T1  Value vaut  9
T2  Value n'est pas assignée
T2  Value vaut  6
MainThread  Value n'est pas assignée
MainThread  Value vaut  3
T1  Value vaut  9
MainThread  Value vaut  3
<_MainThread(MainThread, started 10056)>  -  Se Termine
T2  Value vaut  6


Démonstration que la mémoire local est vraiment par thread ! Du coup, il est "difficile" de partager de la donnée

In [2]:
import threading
from threading import Thread
class CustomString:
    value = " Je suis le message "
    
class StringProcessor(Thread): 
    def __init__(self, message, target = None, name = None, daemon = False):
        super().__init__(target = target, name = name, daemon = daemon)
        #attention msg est une reference
        self.message = message
        #est setter que dans le MainThread attention ne fonctionne pas
        local_data.value = message
        
    def run(self):
        try :
            self.message.value = self.message.value * 2
            print(threading.current_thread().name, "Le résultat est : " + self.message.value, ".")
        except AttributeError:
            print(threading.current_thread().name, "self.message.value n'est pas assignée")
        try :
            print(local_data.value)
        except AttributeError:
            print(threading.current_thread().name, " lcoal_data.value n'est pas assignée")
        
local_data = threading.local()
customstring = CustomString
for i in range(2):
    t = StringProcessor(customstring)
    t.start()


Thread-5 Le résultat est :  Je suis le message  Je suis le message  .
Thread-5  lcoal_data.value n'est pas assignée
Thread-6 Le résultat est :  Je suis le message  Je suis le message  Je suis le message  Je suis le message  .
Thread-6  lcoal_data.value n'est pas assignée


Démonstration qu'il faut passer par les argumetns pour puvoir copier de la mémoire global
auc local par thread pour rendre des obkjcts accesible mais unique par thread 


In [1]:
import threading
from threading import Thread
import copy
class CustomString:
    def __init__(self):
        self.value = "Je suis le message"
    
class StringProcessor(Thread): 
    def __init__(self, message, target = None, name = None, daemon = False):
        super().__init__(target = target, name = name, daemon = daemon)
        self.message = message
        
    def run(self):
        try :
            local_data.message = copy.deepcopy(self.message)
            local_data.message.value = local_data.message.value * 2
            print(threading.current_thread().name, "Le résultat est : " + local_data.message.value, ".")
        except AttributeError:
            print(threading.current_thread().name, "local_data.message.value pas assingee")
        
local_data = threading.local()
customstring = CustomString()
for i in range(2):
    t = StringProcessor(customstring)
    t.start()

Thread-5 Le résultat est : Je suis le messageJe suis le message .
Thread-6 Le résultat est : Je suis le messageJe suis le messageJe suis le messageJe suis le message .


Démonstration local accessible par le globals

In [6]:
from threading import Thread, local, current_thread
from random import randint

def show_value_without_args():
    try:
        val = local_data.value 
    except AttributeError:
        print(current_thread().name, ' - No value yet')
    else:
        print(current_thread().name, ' - value =', val)
        
def worker(): 
    #print(f"{current_thread().name} globals: {list(globals().keys())}")
    show_value_without_args()
    local_data.value = randint(1, 100)
    show_value_without_args()
    
print(current_thread().name, ' - Starting')
local_data = local()

for i in range(2):
    t = Thread(name='W' + str(i), target=worker)
    t.start()
    
print(current_thread().name, ' - Done')

MainThread  - Starting
W0  - No value yet
W0  - value = 86
W1  - No value yet
W1  - value = 54
MainThread  - Done


Exercice sur les Threads Bank Account : Première étape

In [8]:
import threading 
from threading import Thread

def worker():
    print(threading.current_thread().name, "im alive")

worker()
for i in range(0, 5):
    t = Thread(target=worker)
    t.start()

MainThread im alive
Thread-8 (worker) im alive
Thread-9 (worker) im alive
Thread-10 (worker) im alive
Thread-11 (worker) im alive
Thread-12 (worker) im alive


Exercice sur les Threads Bank Account : Deuxieme étape : Thread -> classe

In [10]:
import threading 
from threading import Thread

class TellerThread(Thread):
    def __init__(self, name = None, target = None, daemon = False):
        super().__init__(name = name, target = target, daemon = daemon)
        
    def run(self):
        print(threading.current_thread().name, "im alive")

print(threading.current_thread().name, "im alive")

for i in range(0, 5):
    t = TellerThread()
    t.start()

MainThread im alive
Thread-18 im alive
Thread-19 im alive
Thread-20 im alive
Thread-21 im alive
Thread-22 im alive


Exercice sur les Threads Bank Account : Troiseme étape : Classe BankAccount + afficher la balance dans chaque thread


In [11]:
import threading 
from threading import Thread

class BankAccount:
    def __init__(self, amount):
        self.balance = amount
        
    def deposit(self, amount):
        self.balance += amount
        
    def withdraw(self, amount):
        self.balance -= amount
        
class TellerThread(Thread):
    def __init__(self, band_account, name = None, target = None, daemon = False):
        super().__init__(name = name, target = target, daemon = daemon)
        self.band_account = band_account
        
    def run(self):
        print(threading.current_thread().name, "im alive")
        print(threading.current_thread().name, " - Balance : ", self.band_account.balance) 

print(threading.current_thread().name, "im alive")
bank_account_partage = BankAccount(500)
for i in range(0, 5):
    t = TellerThread(bank_account_partage)
    t.start()

MainThread im alive
Thread-23 im alive
Thread-23  - Balance :  500
Thread-24 im alive
Thread-24  - Balance :  500
Thread-25 im alive
Thread-25  - Balance :  500
Thread-26 im alive
Thread-26  - Balance :  500
Thread-27 im alive
Thread-27  - Balance :  500


Exercice sur les Threads Bank Account : 4e étape : faire plusieurs operation apr thread

In [14]:
import threading 
from threading import Thread
import random
import time

class BankAccount:
    def __init__(self, amount):
        self.balance = amount
        
    def deposit(self, amount):
        self.balance += amount
        
    def withdraw(self, amount):
        self.balance -= amount
        
class TellerThread(Thread):
    def __init__(self, band_account, name = None, target = None, daemon = False):
        super().__init__(name = name, target = target, daemon = daemon)
        self.band_account = band_account
        
    def run(self):
        print(threading.current_thread().name, "im alive")
        print(threading.current_thread().name, " - Balance before transaction : ", self.band_account.balance)
        for i in range(5):
            if random.choice([True, False]):
                amount = randint(1, 100)
                balance_current = self.band_account.balance
                self.band_account.deposit(amount)
                print(threading.current_thread().name," Deposit ", amount , " : ", balance_current, " -> ", self.band_account.balance)
            else:
                amount = randint(1, 100)
                balance_current = self.band_account.balance
                self.band_account.withdraw(amount)
                print(threading.current_thread().name," Withdraw ", amount , " : ",balance_current, " -> ", self.band_account.balance)
            time.sleep(randint(1, 5))

print(threading.current_thread().name, "im alive")
bank_account_partage = BankAccount(500)
for i in range(0, 2):
    t = TellerThread(bank_account_partage)
    t.start()

MainThread im alive
Thread-32 im alive
Thread-32  - Balance before transaction :  500
Thread-32  Withdraw  18  :  500  ->  482
Thread-33 im alive
Thread-33  - Balance before transaction :  482
Thread-33  Withdraw  32  :  482  ->  450
Thread-30  Deposit  10  :  896  ->  906
Thread-31  Deposit  2  :  906  ->  908
Thread-32  Deposit  72  :  450  ->  522
Thread-33  Deposit  60  :  522  ->  582
Thread-33  Deposit  54  :  582  ->  636
Thread-30  Deposit  74  :  908  ->  982
Thread-32  Withdraw  31  :  636  ->  605
Thread-33  Deposit  62  :  605  ->  667
Thread-32  Deposit  78  :  667  ->  745
Thread-30  Deposit  29  :  982  ->  1011
Thread-33  Deposit  51  :  745  ->  796
Thread-32  Deposit  9  :  796  ->  805


Exercice sur les Threads Bank Account : 5e étape : journal de log

In [16]:
import threading 
from threading import Thread
import random
import time

class BankAccount:
    def __init__(self, amount):
        self.balance = amount
        
    def deposit(self, amount):
        self.balance += amount
        
    def withdraw(self, amount):
        self.balance -= amount
        
class TellerThread(Thread):
    def __init__(self, band_account, name = None, target = None, daemon = False):
        super().__init__(name = name, target = target, daemon = daemon)
        self.band_account = band_account
        
    def run(self):
        local_data.log = []
        for i in range(5):
            if random.choice([True, False]):
                amount = randint(1, 100)
                balance_current = self.band_account.balance
                self.band_account.deposit(amount)
                msg = threading.current_thread().name," Deposit ", amount , " : ", balance_current, " -> ", self.band_account.balance
            else:
                amount = randint(1, 100)
                balance_current = self.band_account.balance
                self.band_account.withdraw(amount)
                msg = threading.current_thread().name," Withdraw ", amount , " : ",balance_current, " -> ", self.band_account.balance
            local_data.log.append(msg)
            time.sleep(randint(1, 5))
        print(local_data.log)

print(threading.current_thread().name, "im alive")
bank_account_partage = BankAccount(500)
local_data = threading.local()
for i in range(0, 2):
    t = TellerThread(bank_account_partage)
    t.start()

MainThread im alive
[('Thread-37', ' Deposit ', 22, ' : ', 597, ' -> ', 619), ('Thread-37', ' Withdraw ', 42, ' : ', 699, ' -> ', 657), ('Thread-37', ' Deposit ', 1, ' : ', 691, ' -> ', 692), ('Thread-37', ' Withdraw ', 63, ' : ', 692, ' -> ', 629), ('Thread-37', ' Withdraw ', 23, ' : ', 703, ' -> ', 680)]
[('Thread-36', ' Deposit ', 97, ' : ', 500, ' -> ', 597), ('Thread-36', ' Deposit ', 80, ' : ', 619, ' -> ', 699), ('Thread-36', ' Deposit ', 34, ' : ', 657, ' -> ', 691), ('Thread-36', ' Deposit ', 74, ' : ', 629, ' -> ', 703), ('Thread-36', ' Deposit ', 86, ' : ', 680, ' -> ', 766)]
